In [1]:
# external imports
import numpy as np
import gurobipy as grb

In [20]:
def add_variables(model, n, lb=None, **kwargs):
    """
    Adds n variables to the Gurobi model.
    By default Gurobi sets the lower bouns to zero, here the default is -infinity.

    Arguments
    ----------
    model : instance of gurobipy.Model
        Mathematical program.
    n : int
        Number of optimization variables to be added to the program.
    lb : list of floats
        Lower bound on the optimization variables.

    Returns
    ----------
    x_np : numpy.ndarray
        Optimization variables added to the program, organized in a numpy array.
    """

    # reset the lower bound from 0 to -infinity
    if lb is None:
        lb = [-grb.GRB.INFINITY] * n

    # create optimization variables and organize numpy array
    x = model.addVars(n, lb=lb, **kwargs)
    x_np = np.array([xi for xi in x.values()])

    # update model to be sure that new variables are immediately visible
    model.update()

    return x_np

In [29]:
def add_linear_inequality(model, x, y):
    """
    Adds the linear inequality x <= y to the Gurobi model.

    Arguments
    ----------
    x : numpy array of gurobipy.LinExpr
        Left hand side of the inequality.
    y : numpy array of gurobipy.LinExpr
        Right hand side of the inequality.

    Returns
    ----------
    x_np : numpy.ndarray
        Optimization variables added to the program, organized in a numpy array.
    """
    assert x.size == y.size
    c = [model.addConstr(x[k] - y[k] <= 0.) for k in range(x.size)] # sometimes x is a vector of floats: gurobi raises errors if variables are not in the lhs

    # update model to be sure that new variables are immediately visible
    model.update()

    return c

In [30]:
model = grb.Model()
x = add_variables(model, 3)
c = add_linear_inequality(model, np.eye(3).dot(x), np.zeros(3))

In [31]:
c

[<gurobi.Constr R0>, <gurobi.Constr R1>, <gurobi.Constr R2>]